**Question 1- Find VaR by Parametric Method**

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import fix_yahoo_finance as yf


In [ ]:
stocks= ['DLF.NS','NTPC.NS', 'HDFCBANK.NS']
df=yf.download(stocks, "2017-09-19", "2019-10-02")

initial_investment=1000000



[*********************100%***********************]  3 of 3 downloaded


In [ ]:
df=df[["Adj Close"]]
df.head()

Adj Close                        
                DLF.NS HDFCBANK.NS     NTPC.NS
Date                                          
2017-09-19  184.834198  911.275696  118.306862
2017-09-20  183.913422  910.856995  117.466812
2017-09-21  179.358002  906.053528  117.746834
2017-09-22  168.163269  898.663635  116.206734
2017-09-25  161.475510  886.667358  116.381752

In [ ]:
df_returns= df.pct_change()
df_returns.columns = ['_'.join(col) for col in df_returns.columns.values]
df_returns.rename(columns = {'Adj Close_DLF.NS': 'DLF returns', 'Adj Close_HDFCBANK.NS': 'HDFC returns', 'Adj Close_NTPC.NS':'NTPC returns'}, inplace = True)
df_returns.head()

,DLF returns,HDFC returns,NTPC returns
Date,,,
2017-09-19,NaN,NaN,NaN
2017-09-20,-0.004982,-0.000459,-0.007101
2017-09-21,-0.024769,-0.005274,0.002384
2017-09-22,-0.062416,-0.008156,-0.013080
2017-09-25,-0.039769,-0.013349,0.001506


In [ ]:
avg=df_returns.mean()
wmean= 0.4*avg[0]+0.4*avg[1]+0.2*avg[2]

In [ ]:
cov_matrix = df_returns.cov()
cov_matrix

,DLF returns,HDFC returns,NTPC returns
DLF returns,0.000745,0.000084,0.000084
HDFC returns,0.000084,0.000127,0.000023
NTPC returns,0.000084,0.000023,0.000195


In [ ]:
std_dev= np.sqrt(0.000745*0.16+0.000127*0.16+0.000195*0.04+2*(0.16*0.000084+0.08*0.000084+0.08*0.000023))


In [ ]:
mean_investment = (1 + wmean) * initial_investment           
stdev_investment = std_dev * initial_investment

In [ ]:
conf_level = 0.01
from scipy.stats import norm
cutoff = norm.ppf(conf_level, mean_investment, stdev_investment)

In [ ]:
VaR = initial_investment - cutoff
print("The VaR at 99% Confidence Interval is", VaR)

The VaR at 99% Confidence Interval is 31980.14409847092


In [ ]:
##VaR_99= norm.ppf(1-0.99,wmean,std_dev)
##print("The VaR at 99% confidence level is", VaR_99*initial_investment)
##output is same i.e. VaR= 31980.14409847092

 **Question 2- VaR with HISTORICAL DATA** 

In [ ]:
df_returns["mean"]=0.4*df_returns["DLF returns"]+0.4*df_returns["HDFC returns"]+0.2*df_returns["NTPC returns"]
df_returns["investment_mean"]=(0.4*df_returns["DLF returns"]+0.4*df_returns["HDFC returns"]+0.2*df_returns["NTPC returns"])*initial_investment
df_returns.head()

,DLF returns,HDFC returns,NTPC returns,mean,investment_mean
Date,,,,,
2017-09-19,NaN,NaN,NaN,NaN,NaN
2017-09-20,-0.004982,-0.000459,-0.007101,-0.003597,-3596.560148
2017-09-21,-0.024769,-0.005274,0.002384,-0.011540,-11540.408527
2017-09-22,-0.062416,-0.008156,-0.013080,-0.030845,-30844.634994
2017-09-25,-0.039769,-0.013349,0.001506,-0.020946,-20946.164751


In [ ]:
df_returns.sort_values("investment_mean",ascending=False,inplace=True)
df_returns

,DLF returns,HDFC returns,NTPC returns,mean,investment_mean
Date,,,,,
2019-09-20,0.083012,0.089505,-0.015201,0.065967,65966.559558
2019-05-20,0.056552,0.029741,0.029880,0.040493,40493.172838
2019-08-26,0.049310,0.043210,0.013080,0.039624,39623.875065
2018-10-10,0.066449,0.013681,0.016388,0.035330,35329.997494
2017-11-17,0.072592,0.009874,0.006234,0.034233,34233.493703
...,...,...,...,...,...
2018-10-17,-0.087259,-0.009108,-0.006404,-0.039828,-39827.655004
2019-01-25,-0.111111,-0.002973,-0.012496,-0.048133,-48132.735878
2018-02-02,-0.088163,-0.021671,-0.022774,-0.048489,-48488.581767


In [ ]:
VaR1=df_returns["investment_mean"].quantile(0.01)
VaR1

-35487.87468965388

In [ ]:
print("The VaR is", -Var1)

The VaR is 35487.87468965388


**Question 3: Monte Carlo Simulation**

**For a European Call Option**

In [ ]:
from random import gauss
from math import exp,sqrt
import math

In [ ]:
def mcs(stock_price,implied_vol,rate_of_return,time_to_maturity):
  return stock_price*exp((rate_of_return-(0.5*(implied_vol**2)))*time_to_maturity+implied_vol*sqrt(time_to_maturity)*gauss(0,1))

In [ ]:
def call(stock_price_at_T,strike_price):
  return max(0,stock_price_at_T-strike_price)

In [ ]:
stock_price=14825.35
strike_price=14750
call_implied_vol=0.1498
rate_of_return=0.07
time_to_maturity=1/12
number_of_simulations=90000

In [ ]:
call_payoff=[]


In [ ]:
discount_factor=math.exp(rate_of_return*time_to_maturity)

In [ ]:
for i in range(number_of_simulations):
  stock_price_at_T= mcs(stock_price,call_implied_vol,rate_of_return,time_to_maturity)
  call_payoff.append(call(stock_price_at_T,strike_price))

In [ ]:
call_price= discount_factor*(sum(call_payoff)/number_of_simulations)
print("Call option price is",call_price)

Call option price is 347.61490533585214


The call price obtained from the monte carlo simulation is 347.61 and the current market price for the call option is 412 which means the call option is overpriced. So I will sell this option.

**For a European Put Option**

In [ ]:
def put(stock_price_at_T,strike_price):
  return max(0,strike_price-stock_price_at_T)

In [ ]:
put_implied_vol=0.2105

In [ ]:
put_payoff=[]

In [ ]:
for i in range(number_of_simulations):
  stock_price_at_T= mcs(stock_price,put_implied_vol,rate_of_return,time_to_maturity)
  put_payoff.append(put(stock_price_at_T,strike_price))

In [ ]:
put_price= discount_factor*(sum(put_payoff)/number_of_simulations)
print("Put option price is",put_price)

Put option price is 286.8232758040405


The put price obtained from the monte carlo simulation is 286.82 and the  current market price is 291 which means the put option is overpriced. So I will sell this option.